# Optimal shopping problem

I need to get some goods to met a list of requirements.

There are a ton of goods that can partially satisfy one or more of my requirements.

How many goods do I get? I need to solve an optimization problem.

$$
\begin{array}{lll} 
    \mbox{Minimize} & \sum_{i=1}^{N} c_ix_i & \to \text{Minimize cost of goods}\\
    \mbox{Subject to} & \sum_{i=1}^{N} x_i q_{ij} \geqslant m_j \quad \forall j & \to \text{Fulfilling my "needs" or "requirements"} \\
    & x_i \geqslant 0 \quad \forall i 
\end{array}
$$

Where:

$
\begin{array}{ll} 
    \mbox{$x_i$}    : & \text{Quantity of good $i$ to be obtained} \\
    \mbox{$c_i$}    : & \text{Cost of good $i$} \\
    \mbox{$q_{ij}$} : & \text{Quantity of need $j$ fulfilled for each good $i$ obtained} \\
    \mbox{$m_j$}    : & \text{My total requirement of need $j$ satisfied to fulfill my desires} \\
\end{array}
$

If the total cost is not of our interest we can simple let $c_i = 1, \forall i$, and the problem will minimize the total quantity of goods needed to fullfil the requirements.

In [30]:
import cvxpy as cvx
import numpy as np
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

i = 10   # Type of goods available
j = 5    # Type of needs to be fulfilled

# Random generation of C, Q and M
C = np.random.randint(1,10,i)         # Cost of each good
Q = np.random.choice([0,1,2], (j,i))  # Quantity of each need fulfillef for each good
m = np.random.randint(1,100,(j))      # Requirements of each need

print("Number of goods: i =", i, "\n")
print("Number of needs: j =", j, "\n")
print("Cost of each good: \n C =", C, "\n")
print("Requirements of each need: \n m =", m, "\n")
print("Quantity of each requeriment fulfilled for each good: \n Q =")
print(*Q, sep=' \n')

Number of goods: i = 10 

Number of needs: j = 5 

Cost of each good: 
 C = [9 2 1 9 1 7 5 6 2 1] 

Requirements of each need: 
 m = [94 99 89 85  5] 

Quantity of each requeriment fulfilled for each good: 
 Q =
[2 0 1 2 2 1 0 2 1 0] 
[1 2 2 1 0 0 1 2 2 0] 
[0 1 0 2 2 0 2 1 1 2] 
[0 1 1 0 0 0 0 1 1 2] 
[0 0 1 2 0 2 1 1 1 0]


We will use [CVXPY](https://www.cvxpy.org/) to construct and solve the optimization problem.

We use vector notation:

$
\begin{array}{lll} 
    \mbox{Minimize} & \mathbf{Cx} \\
    \mbox{Subject to} & \mathbf{Qx} \geqslant \mathbf{m} \\
    \mbox{} & \mathbf{x} \geqslant 0
\end{array}
$

In [31]:
# Construct the problem
x = cvx.Variable(i, integer=True)

objective = cvx.Minimize(C*x)
constraints = [Q*x >= M, x >= 0]

prob = cvx.Problem(objective, constraints)

In [32]:
# Solve the problem
sol = prob.solve()

if(prob.status == 'optimal'):
    print("Solución encontrada! \nCosto total =", sol, prob.status)
else:
    print("ERROR: ", prob.status)

Solución encontrada! 
Costo total = 61.99999997577728 optimal


### Verificación de solucíon

In [18]:
x_opt = x.value[np.newaxis].T
print("Cantidad de sacos de cada fertilizante: \n x_optimo = \n", x_opt.astype(int))

Cantidad de sacos de cada fertilizante: 
 x_optimo = 
 [[ 0]
 [ 0]
 [16]
 [39]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]
 [ 0]]


In [229]:
print("Cantidad objetivo de cada químico : \n M =", M, "\n")
print("Cantidad de químico aplicado: \n M_optimo =", Q.dot(x_opt).T)

Cantidad objetivo de cada químico : 
 M = [43 68 65 70 40] 

Cantidad de químico aplicado: 
 M_optimo = [[45. 78. 65. 70. 41.]]


In [230]:
print("Costo total = ", C.dot(x.value))

Costo total =  223.99999998528904
